In [62]:
import numpy as np
import sympy
import csv
from json import loads

lab_number = '4.1'
var_number = 2


def toCopyString(matrix) -> str:
    return 'matrix(' + str(matrix).replace('Matrix([', '[').replace('])', ']')[1:-1].replace(' ', '') + ')'

def fromCopyString(string: str):
    return loads(string.replace('matrix(', '[').replace(')', ']'))

def получить_матрицы_варианта(number):
    filename = f'./{lab_number}_variants.txt'
    with open(filename, 'r') as file:
        variants = csv.reader(file, delimiter=',')
        variant = list(variants)[number - 1][1].split(';')
        A = fromCopyString(variant[0])
        B = fromCopyString(variant[1])
        L = fromCopyString(variant[2])
        C = fromCopyString(variant[3])
    return A, B, L, C

In [63]:
A, B, L, C = получить_матрицы_варианта(var_number)
A = sympy.Matrix(A)
B = sympy.Matrix(B)
L = sympy.Matrix(L)
C = sympy.Matrix(C)

In [68]:
def solve(A, B, L, C):
    # Матрица наблюдаемости
    observability_matrix = sympy.Matrix([(C.T).T, (A.T*C.T).T, (A.T**2*C.T).T]).T

    # Определитель матрицы наблюдаемости
    determinant = observability_matrix.det()
    
    # Желаемые коэффициенты характеристического полинома
    s = sympy.symbols('s')
    characteristic_polynomial = (s - L[0])*(s - L[1])*(s - L[2])
    coefficients = sympy.Matrix(sympy.Poly(characteristic_polynomial, s).all_coeffs()[:0:-1])

    # Матрица K
    K = sympy.Matrix([sympy.symbols(f'K{i}') for i in range(3)])
    det = (s*sympy.eye(3) - A + K*C).det()
    eq = det - characteristic_polynomial
    K = sympy.Matrix(list(sympy.solve(eq, K).values())).n()

    # Матрица наблюдающего устройства
    observing_matrix = A - K*C

    return observability_matrix, determinant, coefficients, K, observing_matrix

In [69]:
solve(A, B, L, C)

(Matrix([
 [0, -8, -4],
 [1,  2, 64],
 [0, -4, -8]]),
 -48,
 Matrix([
 [240],
 [118],
 [ 19]]),
 Matrix([
 [ 211.75],
 [   28.0],
 [-515.75]]),
 Matrix([
 [ 1, -219.75, -3],
 [-8,   -26.0, -4],
 [-5,  516.75,  6]]))

In [70]:
for var_number in range(1, 31):
    A, B, L, C = получить_матрицы_варианта(var_number)
    A = sympy.Matrix(A)
    B = sympy.Matrix(B)
    L = sympy.Matrix(L)
    C = sympy.Matrix(C)
    try:
        observability_matrix, determinant, coefficients, K, observing_matrix = solve(A, B, L, C)

        print(var_number)
        with open(f'./results/{var_number}.txt', 'w') as file:
            file.write(toCopyString(observability_matrix) + '\n')
            file.write(str(determinant) + '\n')
            file.write(toCopyString(coefficients) + '\n')
            file.write(toCopyString(K) + '\n')
            file.write(toCopyString(observing_matrix) + '\n')
    # catch error
    except Exception as e:
        print(f'Error in {var_number}: {e}')   
        

1
2
3
4
5
6
7
8
9
10
11
12
13
14
15
16
17
18
19
20
21
22
23
24
25
26
27
28
29
30
